# Scraping What 1 mnth of law for HK has looked like from INSIDER

## 2.1 Contents<a id='2.1_Contents'></a>
* [2.1 Importing Relevant Tools](#2.1_Importing)
* [2.2 Defining the Request](#2.2_URL)
* [2.3 Grab | Date](#2.4_scrape_date)
* [2.4 Grab | Header](#2.4_scrape_header)
* [2.5 Grab | Content](#2.4_scrape_content)
* [2.6 Clean | Send to DataFrame](#2.5_review)
* [2.7 Save](#2.6_save)


**Importing Relevant Tools**<a id='2.1_Importing'></a>

In [1]:
import json
import pandas as pd
from bs4 import BeautifulSoup as soup
import requests
import csv
import re
import pickle

%reload_ext watermark

In [2]:
#the below needs to be reviewed for all websites; notably the time format

from datetime import date
from datetime import datetime
today = date.today()
d = today.strftime("%m-%d-%y")

For reference.

In [3]:
%watermark -d -t -v -p pandas

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.19.0

pandas: 1.1.3



**Defining the Request**<a id='2.2_URL'></a>

In [4]:
url = 'https://www.businessinsider.com/hong-kong-china-national-security-law-one-month-what-changed-2020-7'.format(d)
url

'https://www.businessinsider.com/hong-kong-china-national-security-law-one-month-what-changed-2020-7'

In [5]:
html = requests.get(url)

In [63]:
bsobj = soup(html.content,'lxml')
# bsobj

**Grab | Date**<a id='2.4_scrape_date'></a>

In [32]:
for date in bsobj.findAll('div',{'data-timestamp':'2020-08-02T07:10:00Z'}):
    # bsObj.select_one
    print(date.text.strip())

2020-08-02T07:10:00Z


Needs work below.

**Grab | Header**<a id='2.4_scrape_header'></a>

In [34]:
for title in bsobj.findAll("h1"):
    print(format(title.text))

Teenage arrests, blank protest signs, and a key election postponed: What one month of China's new national security law for Hong Kong has looked like


Confirmed that's the title.

**Grab | Content**<a id='2.4_scrape_content'></a>

In [93]:
for news in bsobj.findAll('div',{'class':'content-lock-content'}):
    print(news.text.strip())

Thursday, July 30, marked one month since China imposed a new national-security law on Hong Kong.A lot has happened in a very short time.Vocal activists fled to safety abroad, political organizations disbanded, and pro-democracy candidates were banned from running in September's legislative election (which was also postponed on Friday).But pro-democracy activists have vowed to fight on, with some finding new ways to protest without falling afoul of the law, and others planning a parliament in exile.Elsewhere, China and Britain have exchanged barbs after the UK promised to give 3 million Hong Kongers a path to citizenship.Visit Business Insider's homepage for more stories.

Thursday, July 30, marked one month since China's new national security law came into force in Hong Kong.From June 30, China has wielded the power to define and punish "separatism, subversion, terrorism and foreign interference" in the city as it sees fit. This means that anti-China sentiment — be it waving banners, 

That's the bottom but a lot to remove.

**Clean | Send to DataFrame**<a id='2.5_review'></a>

First the date from string to a datetime object.

In [70]:
for day in bsobj.find('div',{'data-timestamp':'2020-08-02T07:10:00Z'}):
    day_pub = day.strip()

In [71]:
day_pub

'2020-08-02T07:10:00Z'

In [77]:
day_pub = re.sub('T07:10:00Z','',day_pub)
day_pub = datetime.strptime(day_pub, '%Y-%m-%d').date()
day_pub

datetime.date(2020, 8, 2)

In [78]:
df_date = pd.DataFrame([day_pub])

In [79]:
type(df_date)

pandas.core.frame.DataFrame

In [80]:
df_date

,0
0,2020-08-02


Now the title.

In [81]:
for title_s in bsobj.findAll("h1"):
    title_list = format(title_s.text)

In [82]:
df_title = pd.DataFrame([title_list])

In [83]:
df_title

,0
0,"Teenage arrests, blank protest signs, and a ke..."


In [84]:
type(df_title)

pandas.core.frame.DataFrame

These two items are manually added.

In [89]:
country = 'US'
df_country = pd.DataFrame([country])
source = 'Insider'
df_source = pd.DataFrame([source])

Finally, the news.

In [94]:
news1 = []
for news in bsobj.findAll('div',{'class':'content-lock-content'}):
    news1.append(news.text.strip())

In [95]:
news1

['Thursday, July 30, marked one month since China imposed a new national-security law on Hong Kong.A lot has happened in a very short time.Vocal activists fled to safety abroad, political organizations disbanded, and pro-democracy candidates were banned from running in September\'s legislative election (which was also postponed on Friday).But pro-democracy activists have vowed to fight on, with some finding new ways to protest without falling afoul of the law, and others planning a parliament in exile.Elsewhere, China and Britain have exchanged barbs after the UK promised to give 3 million Hong Kongers a path to citizenship.Visit Business Insider\'s homepage for more stories.\n\nThursday, July 30, marked one month since China\'s new national security law came into force in Hong Kong.From June 30, China has wielded the power to define and punish "separatism, subversion, terrorism and foreign interference" in the city as it sees fit.\xa0This means that anti-China sentiment — be it waving

In [96]:
len(news1)

1

In [23]:
df_news = pd.DataFrame()

In [24]:
df_news['article_body'] = news1

In [25]:
df_news.head(2)

,article_body
0,China has passed a wide-ranging new security l...
1,"Critics have called it ""the end of Hong Kong"" ..."


In [26]:
df_news['article_body'] = df_news.article_body.str.cat(sep='')

In [27]:
df_news = df_news.article_body[0]

In [28]:
df_news = df_news.replace(r'\\?','')

In [29]:
df_news = pd.DataFrame([df_news])

In [30]:
type(df_news)

pandas.core.frame.DataFrame

In [31]:
df_news.columns = ['Article']

In [32]:
df_news.head()

,Article
0,China has passed a wide-ranging new security l...


**Bringing it together.**<a id='2.5_bit'></a>

In [33]:
df_1_bbc = pd.concat([df_date,df_source,df_country,df_title,df_news],axis = 1, ignore_index=False)

In [34]:
df_1_bbc.columns = ['date','source','country','title','article']

In [35]:
df_1_bbc.head()

,date,source,country,title,article
0,2020-06-30,BBC,UK,Hong Kong security law: What is it and is it w...,China has passed a wide-ranging new security l...


**Saving**<a id='2.6_save'></a>

In [36]:
cd

C:\Users\rands


Saving it to Excel.

In [37]:
# df = pd.DataFrame(reviewlist)

# index=False below so that we don't get the dataframe index on the side; we just use the excel index
df_1_bbc.to_excel('./_Capstone_Two_NLP/data/_news/bbc_1.xlsx', index=False)

print('Complete')

Complete
